In [1]:
import numpy as np
import pandas as pd
import yfinance
import statsmodels.formula.api as smf
import statsmodels.api as smi
from table_making import summary_col
import matplotlib.pyplot as plt
import re
import os

In [2]:
if not os.path.exists("tables/fsi"):
    os.makedirs("tables/fsi")

# Load Data

In [3]:
fsi = pd.read_csv("data/fsi.csv")
fsi = fsi.rename(columns={"Date": "date", 
                          "OFR FSI": "FSI", "Credit": "credit", "Equity valuation": "equity",
                          "Safe assets": "safe_assets", "Funding": "funding", "Volatility": "real_vol"})
fsi["date"] = pd.to_datetime(fsi["date"])
fsi.head()

,date,FSI,credit,equity,safe_assets,funding,real_vol,United States,Other advanced economies,Emerging markets
0,2000-01-03,2.140,0.540,-0.051,0.670,0.472,0.509,1.769,0.521,-0.150
1,2000-01-04,2.421,0.604,0.079,0.627,0.550,0.561,2.084,0.474,-0.137
2,2000-01-05,2.297,0.575,0.080,0.653,0.501,0.488,2.023,0.406,-0.132
3,2000-01-06,2.292,0.546,0.082,0.581,0.566,0.517,2.064,0.355,-0.127
4,2000-01-07,2.005,0.662,-0.011,0.519,0.565,0.270,1.769,0.367,-0.131


In [4]:
spx_df = pd.read_parquet("data/spx_vol_surface_history_full_data_23.parquet").reset_index().rename(columns={"index": "date"})
spx_df2 = pd.merge(spx_df, fsi, how="left", on=["date"])
spx_df2 = spx_df2.dropna()
spx_df2.head()

,date,ttm_one_month_moneyness_pt_seven,ttm_one_month_moneyness_pt_eightfive,ttm_one_month_moneyness_pt_one,ttm_one_month_moneyness_pt_oneonefive,ttm_one_month_moneyness_pt_onethree,ttm_three_month_moneyness_pt_seven,ttm_three_month_moneyness_pt_eightfive,ttm_three_month_moneyness_pt_one,ttm_three_month_moneyness_pt_oneonefive,...,observation,FSI,credit,equity,safe_assets,funding,real_vol,United States,Other advanced economies,Emerging markets
0,2000-01-03,6.194064e-02,0.312185,0.204530,0.130964,0.000000e+00,0.260440,0.275718,0.221527,0.168832,...,114.0,2.140,0.540,-0.051,0.670,0.472,0.509,1.769,0.521,-0.150
1,2000-01-04,1.363700e-01,0.320018,0.240132,0.144813,5.976089e-17,0.416818,0.300325,0.230710,0.182980,...,114.0,2.421,0.604,0.079,0.627,0.550,0.561,2.084,0.474,-0.137
2,2000-01-05,9.418156e-18,0.342766,0.229619,0.144372,2.613037e-17,0.097694,0.295088,0.234404,0.188767,...,110.0,2.297,0.575,0.080,0.653,0.501,0.488,2.023,0.406,-0.132
3,2000-01-06,1.702120e-16,0.154186,0.186524,0.144133,0.000000e+00,0.038355,0.251156,0.235466,0.164924,...,107.0,2.292,0.546,0.082,0.581,0.566,0.517,2.064,0.355,-0.127
4,2000-01-07,7.347157e-01,0.400736,0.205539,0.125236,4.878086e-17,0.393377,0.303629,0.220231,0.170690,...,128.0,2.005,0.662,-0.011,0.519,0.565,0.270,1.769,0.367,-0.131


In [5]:
spx_df2.to_parquet("data/spx_with_fsi.parquet", index=False)

# Function Def

In [6]:
def latex_format(sm):
    ltx = sm.as_latex()
    ltx = re.sub(r" +", " ", ltx)
    ltx = ltx.replace(" & FSI & credit & equity & safe\_assets & funding & real\_vol \\\\\n\\hline", """ & FSI & credit & equity & safe assets & funding & real volatility \\\\
 & (1) & (2) & (3) & (4) & (5) & (6) \\\\
\cmidrule(lr){2-2} \cmidrule(lr){3-3} \cmidrule(lr){4-4} \cmidrule(lr){5-5} \cmidrule(lr){6-6} \cmidrule(lr){7-7}""")
    ltx = ltx.replace("iv\_atm\_std", "$\sigma(\\text{iv}_{\\text{ATM}})$")
    ltx = ltx.replace("iv\_atm\_mean", "$\mu(\\text{iv}_{\\text{ATM}})$")
    ltx = ltx.replace("R-squared Adj.", "\\hline\n$R^2$")
    ltx = ltx.replace("\\$N\\$", "$N$")
    ltx = ltx.replace("""lllllll""", """lcccccc""")
    ltx = ltx.replace("cc}\n\\hline", "cc}\n\\hline\\hline")
    ltx = ltx.replace("\\begin{table}\n\\caption{}\n\\label{}\n\\begin{center}\n", "")
    ltx = ltx.replace("\\end{center}\n\\end{table}", "")
    return ltx

# Regression - no ex feats

## std

In [7]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27_simple/model_0_gen3.npy") # no ex feats
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/no_ex_overall_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [8]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27_simple/model_0_gen3.npy") # no ex feats
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/no_ex_overall_mean.tex", "w") as f:
    f.write(latex_format(sm))

# Regression - no loss

## std

In [9]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/no_loss_overall_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [10]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/no_loss_overall_mean.tex", "w") as f:
    f.write(latex_format(sm))

# Regression - loss

## std

In [11]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27/model_1_gen3.npz")["surfaces"]
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/loss_overall_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [12]:
spx_df = pd.read_parquet("data/spx_with_fsi.parquet")
simulated_surfaces = np.load("test_spx/2023_08_27/model_1_gen3.npz")["surfaces"]
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, 
["date", "FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["FSI", "credit", "equity", "safe_assets", "funding", "real_vol"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/fsi/loss_overall_mean.tex", "w") as f:
    f.write(latex_format(sm))